## **Extraer datos Generales por hora**

In [0]:
#codigo para guardar todos los insights necesarios para el analisis de datos de SOLO los anuncios
import requests
import pandas as pd
import os

# Leer el token desde el archivo CSV previamente guardado
df_TokenLargoPlazo = pd.read_csv("/dbfs/mnt/Token_LargoPlazo/long_lived_token.csv")
access_token = df_TokenLargoPlazo["access_token"].iloc[0]

# Reemplaza este valor con tu ad_account_id (ID de la cuenta publicitaria de Meta)
ad_account_id = "1952584805183463"

# Crear el directorio '/dbfs:/mnt/processed' si no existe
processed_dir = '/dbfs/mnt/processed/Ads_General_Por_Hora'
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

# Función para obtener todos los conjuntos de anuncios de todas las campañas
def obtener_conjuntos_anunciosMeta(access_token, ad_account_id):
    url = f"https://graph.facebook.com/v13.0/act_{ad_account_id}/adsets"
    params = {
        "access_token": access_token,
        "fields": "id,name,status"  # Especifica las columnas que deseas obtener
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()['data']
        df_conjuntosMeta = pd.DataFrame(data)
        return df_conjuntosMeta
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

# Función para obtener el alcance de un anuncio mediante el endpoint de insights
def obtener_alcance_anuncio(access_token, ad_id):
    url = f"https://graph.facebook.com/v13.0/{ad_id}/insights"
    params = {
        "access_token": access_token,
        "fields": "ad_id,ad_name,adset_name,campaign_name,date_start,date_stop,reach,impressions,frequency,spend,clicks,cost_per_ad_click,inline_link_clicks,conversion_rate_ranking,cpc,cpp,cpm,actions,ad_click_actions,quality_ranking,conversions,buying_type",  # Especifica las métricas que deseas obtener
        "breakdowns": "hourly_stats_aggregated_by_advertiser_time_zone",
        "level": "ad"  # Nivel de la métrica, en este caso es a nivel de anuncio
    }

    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()['data']
        df_insights = pd.DataFrame(data)
        return df_insights
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

# Obtener todos los conjuntos de anuncios
df_conjuntos = obtener_conjuntos_anunciosMeta(access_token, ad_account_id)

if df_conjuntos is not None:
    all_insights = []
    # Obtener y guardar el alcance y otras métricas para cada anuncio en los conjuntos de anuncios
    for adset_id in df_conjuntos['id']:
        url = f"https://graph.facebook.com/v13.0/{adset_id}/ads"
        params = {
            "access_token": access_token,
            "fields": "id,name"  # Obtener solo los IDs de los anuncios
        }
        response = requests.get(url, params=params)
        if response.status_code == 200:
            ads_data = response.json()['data']
            for ad in ads_data:
                ad_id = ad['id']
                df_anuncio_insights = obtener_alcance_anuncio(access_token, ad_id)
                if df_anuncio_insights is not None:
                    all_insights.append(df_anuncio_insights)

    # Unir todos los DataFrames de insights en uno solo
    if all_insights:
        df_all_insights = pd.concat(all_insights, ignore_index=True)
        # Guardar todos los insights en un solo CSV
        df_all_insights.to_csv(f"{processed_dir}/anuncios_insights_general_por_hora.csv", index=False)
        print("Todos los insights de los anuncios generales por hora guardados exitosamente.")
    else:
        print("No se encontraron insights de anuncios.")
else:
    print("No se encontraron conjuntos de anuncios o limite de solicitud excedido.")
#revisar cuales archivos existen dentro de la ruta
dbutils.fs.ls("/mnt/processed/Ads_General_Por_Hora")

Todos los insights de los anuncios generales por hora guardados exitosamente.


[FileInfo(path='dbfs:/mnt/processed/Ads_General_Por_Hora/anuncios_insights_general_por_hora.csv', name='anuncios_insights_general_por_hora.csv', size=367228, modificationTime=1731510034000)]

In [0]:
#revisar el contenido de cada columna
import pandas as pd
#crear dataframe para leer el csv
df = pd.read_csv("/dbfs/mnt/processed/Ads_General_Por_Hora/anuncios_insights_general_por_hora.csv")

# Ver las columnas del DataFrame
display(df.columns)
display(df)

Index(['ad_id', 'ad_name', 'adset_name', 'campaign_name', 'date_start',
       'date_stop', 'impressions', 'spend', 'clicks', 'inline_link_clicks',
       'cpc', 'cpm', 'actions', 'buying_type',
       'hourly_stats_aggregated_by_advertiser_time_zone'],
      dtype='object')

ad_id,ad_name,adset_name,campaign_name,date_start,date_stop,impressions,spend,clicks,inline_link_clicks,cpc,cpm,actions,buying_type,hourly_stats_aggregated_by_advertiser_time_zone
120211362612890669,Test_Ad_BosqueDesertAlta,Test_Group_Ads_Generales,Test_All_Nov_Camp,2024-10-14,2024-11-12,1183,50.69,18,7.0,2.816111,42.84869,"[{'action_type': 'post_engagement', 'value': '8'}, {'action_type': 'page_engagement', 'value': '8'}, {'action_type': 'post_reaction', 'value': '1'}, {'action_type': 'link_click', 'value': '7'}]",AUCTION,10:00:00 - 10:59:59
120211362612890669,Test_Ad_BosqueDesertAlta,Test_Group_Ads_Generales,Test_All_Nov_Camp,2024-10-14,2024-11-12,216,7.33,9,2.0,0.814444,33.935185,"[{'action_type': 'post_engagement', 'value': '2'}, {'action_type': 'page_engagement', 'value': '2'}, {'action_type': 'link_click', 'value': '2'}]",AUCTION,11:00:00 - 11:59:59
120211362612890669,Test_Ad_BosqueDesertAlta,Test_Group_Ads_Generales,Test_All_Nov_Camp,2024-10-14,2024-11-12,247,9.44,8,1.0,1.18,38.218623,"[{'action_type': 'post_engagement', 'value': '3'}, {'action_type': 'page_engagement', 'value': '3'}, {'action_type': 'post', 'value': '1'}, {'action_type': 'post_reaction', 'value': '1'}, {'action_type': 'link_click', 'value': '1'}]",AUCTION,12:00:00 - 12:59:59
120211362612890669,Test_Ad_BosqueDesertAlta,Test_Group_Ads_Generales,Test_All_Nov_Camp,2024-10-14,2024-11-12,284,8.36,7,1.0,1.194286,29.43662,"[{'action_type': 'post_engagement', 'value': '1'}, {'action_type': 'page_engagement', 'value': '1'}, {'action_type': 'link_click', 'value': '1'}]",AUCTION,13:00:00 - 13:59:59
120211362612890669,Test_Ad_BosqueDesertAlta,Test_Group_Ads_Generales,Test_All_Nov_Camp,2024-10-14,2024-11-12,215,9.02,6,2.0,1.503333,41.953488,"[{'action_type': 'post_engagement', 'value': '2'}, {'action_type': 'page_engagement', 'value': '2'}, {'action_type': 'link_click', 'value': '2'}]",AUCTION,14:00:00 - 14:59:59
120211362612890669,Test_Ad_BosqueDesertAlta,Test_Group_Ads_Generales,Test_All_Nov_Camp,2024-10-14,2024-11-12,167,4.26,3,1.0,1.42,25.508982,"[{'action_type': 'post_engagement', 'value': '1'}, {'action_type': 'page_engagement', 'value': '1'}, {'action_type': 'link_click', 'value': '1'}]",AUCTION,15:00:00 - 15:59:59
120211362612890669,Test_Ad_BosqueDesertAlta,Test_Group_Ads_Generales,Test_All_Nov_Camp,2024-10-14,2024-11-12,381,12.71,16,10.0,0.794375,33.35958,"[{'action_type': 'post_engagement', 'value': '10'}, {'action_type': 'page_engagement', 'value': '11'}, {'action_type': 'like', 'value': '1'}, {'action_type': 'link_click', 'value': '10'}]",AUCTION,16:00:00 - 16:59:59
120211362612890669,Test_Ad_BosqueDesertAlta,Test_Group_Ads_Generales,Test_All_Nov_Camp,2024-10-14,2024-11-12,256,8.73,4,1.0,2.1825,34.101563,"[{'action_type': 'post_engagement', 'value': '1'}, {'action_type': 'page_engagement', 'value': '1'}, {'action_type': 'link_click', 'value': '1'}]",AUCTION,17:00:00 - 17:59:59
120211362612890669,Test_Ad_BosqueDesertAlta,Test_Group_Ads_Generales,Test_All_Nov_Camp,2024-10-14,2024-11-12,359,14.69,10,2.0,1.469,40.91922,"[{'action_type': 'post_engagement', 'value': '3'}, {'action_type': 'page_engagement', 'value': '3'}, {'action_type': 'post_reaction', 'value': '1'}, {'action_type': 'link_click', 'value': '2'}]",AUCTION,18:00:00 - 18:59:59
120211362612890669,Test_Ad_BosqueDesertAlta,Test_Group_Ads_Generales,Test_All_Nov_Camp,2024-10-14,2024-11-12,400,13.89,9,2.0,1.543333,34.725,"[{'action_type': 'post_engagement', 'value': '3'}, {'action_type': 'page_engagement', 'value': '3'}, {'action_type': 'post_reaction', 'value': '1'}, {'action_type': 'link_click', 'value': '2'}]",AUCTION,19:00:00 - 19:59:59
